In [1]:
import pandas as pd
import nltk
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv ("classifier1.csv",on_bad_lines='skip')
df
df.head()

,paragraph,classe
0,if you live in the european region whatsapp ir...,False
1,whatsapp legal info,False
2,if you live outside the european region whatsa...,False
3,for example our privacy policy talks about wha...,False
4,we are one of the facebook companies you can l...,False


In [3]:
# turn the classes and paragraphs into lists 
classes = df['classe'].tolist()
paragraphs = df['paragraph'].tolist()

In [4]:
classes [3], paragraphs [3]

(False,
 'for example our privacy policy talks about what information we collect and how this affects you it also explains the steps we take to protect your privacy like building our services so delivered messages arent stored by us and giving you control over who you communicate with on our services')

In [5]:
classe_legend = {"True" : 1, "False" : 0}
classe_legend_reverted = {f"{v}": k for k,v in classe_legend.items()}
classe_legend

{'True': 1, 'False': 0}

In [6]:
# Il faut mieux diviser le dataset une seule fois et stocker les deux datasets dans deux fichiers séparés
# Puisque si vous appliquez ça à chaque fois les échantillons seront différents et donc
# vous ne pouvez pas comparer entre les différents modèles si vous appliquez des entraînement séparés
# X1_train, X1_test, y_train, y_test = train_test_split(paragraphs, classes, test_size=0.2, random_state=42)
X1_train, X1_test, y_train, y_test = train_test_split(df["paragraph"],df["classe"],test_size=0.2, random_state=42)


count_vectorizer = CountVectorizer()
X_train = count_vectorizer.fit_transform(X1_train)
X_test = count_vectorizer.transform(X1_test)

In [7]:
# convert categoricals
from sklearn.preprocessing import LabelEncoder

encoders = {}
# for column in ['classe']:
categorical_convert = LabelEncoder()
df.classe = categorical_convert.fit_transform(df.classe.values)
encoders['classe'] = categorical_convert
    
encoders

{'classe': LabelEncoder()}

In [8]:
#entrainement Naive Bayes
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

MultinomialNB()

In [9]:
#test Naive Bayes
Y_naive_bayes = naive_bayes.predict(X_test)

In [10]:
print(classification_report(y_test, Y_naive_bayes))

              precision    recall  f1-score   support

       False       0.70      0.64      0.67        33
        True       0.82      0.86      0.84        63

    accuracy                           0.78        96
   macro avg       0.76      0.75      0.75        96
weighted avg       0.78      0.78      0.78        96



In [11]:
# Pour naive bayes, la prédiction doit passer par le même pipeline
# Dans la version précédente, vous avez utilisé un autre pipeline que celui 
# utilisé dans l'entraînement

def predict(text_str):
    x_input = count_vectorizer.transform([text_str])
    return naive_bayes.predict(x_input)[0]

In [12]:
predict("we may share personal data with card networks and payment processors")

True

In [13]:
predict("if the new version reduces your rights or increases your responsibilities well post it on the policy updates or privacy statement page of our website at least 21 days before it becomes effective")

False

In [ ]:
# input_data{}

# data = count_vectorizer.transform([word.lower() for word in data])

In [14]:
import joblib
joblib.dump(naive_bayes, "./naive_bayes.joblib", compress=True)
joblib.dump(count_vectorizer, "./count_vectorizer.joblib", compress=True)

['./count_vectorizer.joblib']

In [15]:
modelReload = joblib.load("naive_bayes.joblib")
modelReload

MultinomialNB()

In [16]:
vectorizer_to_train_data = joblib.load("count_vectorizer.joblib")
vectorizer_to_train_data

CountVectorizer()

In [27]:
from scipy.sparse import csr_matrix
str = ['make it easy for you to find and connect with others for instance if you let us access your contacts we can suggest connections with people you may know']
x_input = vectorizer_to_train_data.transform([word.lower() for word in str])
# print (x_input [0])
csr_matrix.toarray(x_input)
print (csr_matrix.toarray(x_input)[0])
modelReload.predict(csr_matrix.toarray(x_input))[0]

# texts = [[word.lower() for word in text.split()] for text in data]

[0 0 0 ... 0 0 0]


True